In [7]:
import pandas as pd


df = pd.read_parquet('all_funda_russell3000.parquet')

def calculate_profitability_metrics(df):
    """
    Calculate required profitability metrics and add them as new columns to the DataFrame.

    Metrics:
    1. Gross Profits Over Assets (GPOA)
    2. Return on Equity (ROE)
    3. Return on Assets (ROA)
    4. Cash Flow Over Assets (CFOA)
    5. Gross Margin (GMAR)
    6. Low Accruals (ACC)

    Parameters:
    df (pd.DataFrame): DataFrame containing historical quarterly data.

    Returns:
    pd.DataFrame: DataFrame with new columns for the calculated metrics.
    """
    # Ensure numeric columns are properly handled
    df = df.copy()
    
    # Calculate GPOA (Gross Profits Over Assets)
    if {'grossProfit', 'totalAssets'}.issubset(df.columns):
        df['GPOA'] = df['grossProfit'] / df['totalAssets']
    else:
        df['GPOA'] = None  # Mark as NaN if required columns are missing

    # Calculate ROE (Return on Equity)
    if {'netIncome', 'totalStockholdersEquity'}.issubset(df.columns):
        df['ROE'] = df['netIncome'] / df['totalStockholdersEquity']
    else:
        df['ROE'] = None

    # Calculate ROA (Return on Assets)
    if {'netIncome', 'totalAssets'}.issubset(df.columns):
        df['ROA'] = df['netIncome'] / df['totalAssets']
    else:
        df['ROA'] = None

    # Calculate CFOA (Cash Flow Over Assets)
    if {'operatingCashFlowPerShare', 'totalAssets', 'weightedAverageShsOut'}.issubset(df.columns):
        df['operatingCashFlow'] = df['operatingCashFlowPerShare'] * df['weightedAverageShsOut']
        df['CFOA'] = df['operatingCashFlow'] / df['totalAssets']
    else:
        df['CFOA'] = None

    # Calculate GMAR (Gross Margin)
    if {'grossProfit', 'revenue'}.issubset(df.columns):
        df['GMAR'] = (df['grossProfit'] / df['revenue']) * 100
    else:
        df['GMAR'] = None

    # Calculate ACC (Low Accruals)
    if {'operatingCashFlow', 'netIncome'}.issubset(df.columns):
        df['ACC'] = 1 - (df['operatingCashFlow'] / df['netIncome'])
    else:
        df['ACC'] = None

    # Drop intermediate calculation columns if added
    if 'operatingCashFlow' in df.columns:
        df.drop(columns=['operatingCashFlow'], inplace=True)

    return df


df = calculate_profitability_metrics(df.loc[0:500])
df[['symbol', 'date', 'calendarYear', 'period','GPOA','ROE','returnOnEquity','ROA','returnOnAssets','CFOA', 'GMAR','grossProfitMargin','ACC']].head()
# df.to_csv('test1.csv', index=False)

,symbol,date,calendarYear,period,GPOA,ROE,returnOnEquity,ROA,returnOnAssets,CFOA,GMAR,grossProfitMargin,ACC
0,A,2024-07-31,2024.0,Q3,0.080029,0.047772,0.047772,0.025646,0.025646,0.041106,55.766793,0.557668,-0.602837
1,A,2024-04-30,2024.0,Q2,0.078850,0.049565,0.049565,0.028371,0.028371,0.030674,54.418309,0.544183,-0.081169
2,A,2024-01-31,2024.0,Q1,0.083394,0.056238,0.056238,0.031787,0.031787,0.044300,55.066345,0.550663,-0.393678
3,A,2023-10-31,2023.0,Q4,0.085013,0.081266,0.081266,0.044133,0.044133,0.047942,54.206161,0.542062,-0.086316
4,A,2023-07-31,2023.0,Q3,0.061639,0.019971,0.019971,0.010398,0.010398,0.052646,39.354067,0.393541,-4.063063
